In [3]:
from IPython.core.display import display, HTML
from h5py import File
import pandas as pd
import numpy as np
from pathlib import Path
import os
root_path = Path(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
print(root_path)
os.chdir(str(root_path))
from FIBI.data.analyse_cpp_results.clustering.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class
import json
from itertools import product

C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche


C:\Users\robin\AppData\Local\Temp\ipykernel_20292\2991227904.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Load data

In [4]:
data = []
with File("data/data_algo_same/table1_exp_clust.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data.append(dico_value)
df = pd.DataFrame(data)


In [5]:
display(df.head())

,key_dataset,NUM_POINTS,NUM_DIM,POINTS_DISTR,NUM_CLUST,CLUST_IMPR,IMPR_CLASS,IT_ORDER,SEED_POINTS,SEED_ASSIGN,init_cost,final_cost,num_iter,num_iter_glob,duration
0,"0,100,0,0,0,16,0,1",100,2,0,16,random,BICL,CURR,0,0,1.0,0.0,0,132861,0.007251
1,"0,100,0,0,0,2,0,1",100,2,0,2,random,BICL,CURR,0,0,1.0,0.0,0,151499,0.090928
2,"0,100,0,0,0,32,0,1",100,2,0,32,random,BICL,CURR,0,0,1.0,0.0,0,104430,0.002527
3,"0,100,0,0,0,4,0,1",100,2,0,4,random,BICL,CURR,0,0,1.0,0.0,0,150420,0.041167
4,"0,100,0,0,0,64,0,1",100,2,0,64,random,BICL,CURR,0,0,1.0,0.0,0,63152,0.000533


## Verify that we have all of the required results

In [6]:
def check_print(cond:bool):
    if cond:
        print("✅")
    else:
        print("❌")

Number of points:

Expected: [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]

Found:

In [7]:
expected = [20,30,40,50,60,70,80,90,100]#,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
num_points = list(df[str(DataCategory.NUM_POINTS)].unique())
num_points.sort()
print(num_points)
check_print(set(num_points) == set(expected))

[20, 30, 40, 50, 60, 70, 80, 90, 100]
✅


Number of different point and assign seed per number of point x algorithm

Expected:

- for n in [20, 30, ..., 150] : 1000
- for n in [200, 250, ..., 500] : 100
- for n in [500, 600, ..., 1000] : 100

Found:

In [8]:
groups_counts = df.groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS),str(DataCategory.NUM_CLUST)]).count().reset_index()
num_points = [20,30,40,50,60,70,80,90,100]#,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
expected = {n: 1000 if n<=150 else 100 for n in num_points}
cond = True

for n in expected:
    for impr_class in corresp_impr_class.values():
        df_tmp = groups_counts.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        if len(df_tmp) == 0:
            print(f"{n} {impr_class}")
            cond = False
            break
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_POINTS)] == expected[n]
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_ASSIGN)] == expected[n]
check_print(cond)
print(len(df))

✅
94000


## Statistics

### Computed statistics

In [43]:
df_table1_src = df[[
    str(DataCategory.NUM_POINTS),
    str(DataCategory.NUM_CLUST),
    str(DataCategory.IMPR_CLASS),
    str(DataCategory.final_cost),
    str(DataCategory.num_iter),
    str(DataCategory.num_iter_glob),
    str(DataCategory.duration),
    ]]
df_table1 = df_table1_src.copy().groupby([str(DataCategory.NUM_POINTS),str(DataCategory.NUM_CLUST),str(DataCategory.IMPR_CLASS)]).mean().reset_index()
# df_table1 = df_table1.query(f"{str(DataCategory.IMPR_CLASS)} != 'BICL'")
df_table1.sort_values(by=[str(DataCategory.NUM_POINTS),str(DataCategory.NUM_CLUST),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table1.reset_index(drop=True, inplace=True)
df_table1_str = df_table1.to_html(formatters={
    str(DataCategory.NUM_POINTS): '{:d}'.format,
    str(DataCategory.NUM_POINTS): '{:d}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost): '{:.2f}'.format,
    str(DataCategory.num_iter): '{:.0f}'.format,
    str(DataCategory.num_iter_glob): '{:.0f}'.format,
    str(DataCategory.duration): '{:.4e}'.format,
})
display(HTML(df_table1_str))

,NUM_POINTS,NUM_CLUST,IMPR_CLASS,final_cost,num_iter,num_iter_glob,duration
0,20,2,BICL,499.50,500,30032,1.6600e-02
1,30,2,BICL,16499.50,16500,46771,2.6759e-02
2,40,2,BICL,32499.50,32500,63384,3.6687e-02
3,50,2,BICL,48499.50,48500,79732,4.6602e-02
4,60,2,BICL,64499.50,64500,96944,5.7139e-02
5,70,2,BICL,80499.50,80500,113702,6.7235e-02
6,80,2,BICL,96499.50,96500,129884,7.7430e-02
7,90,2,BICL,112499.50,112500,146488,8.7531e-02
8,100,2,BICL,128499.50,128500,162987,9.7624e-02
9,110,2,BICL,144499.50,144500,180099,1.0810e-01


## Export results for visualization

In [17]:
computed = df_table1_src.copy()

computed = computed[["NUM_POINTS","NUM_CLUST","IMPR_CLASS","final_cost","num_iter","duration"]]
computed["source"] = "computed"
new_df = computed
display(new_df)

,NUM_POINTS,NUM_CLUST,IMPR_CLASS,final_cost,num_iter,duration,source
0,20,2,BICL,0.0,0,0.013345,computed
1,20,2,BICL,1.0,1,0.015034,computed
2,20,2,BICL,10.0,10,0.013450,computed
3,20,2,BICL,100.0,100,0.013579,computed
4,20,2,FICL,1000.0,1000,0.020299,computed
...,...,...,...,...,...,...,...
178795,80,4,FICL,99995.0,99995,0.034305,computed
178796,80,4,FICL,99996.0,99996,0.032720,computed
178797,80,4,FICL,99997.0,99997,0.029851,computed
178798,80,4,FICL,99998.0,99998,0.029078,computed


In [18]:
def q1(x):
    return x.quantile(0.25)

def median(x):
    return x.quantile(0.50)
def q3(x):
    return x.quantile(0.75)
f = ["mean","std","min",q1,median,q3,"max"]
df_avg = new_df.groupby(["NUM_POINTS","NUM_CLUST","IMPR_CLASS","source"]).agg(f)
df_avg.reset_index(inplace=True)
df_avg.fillna(-1,inplace=True)
Lout = []
for elem in df_avg.to_dict(orient="records"):
    dico = {}
    for k,v in elem.items():
        if k[1] == '':
            dico[k[0]] = v
        else:
            if k[0] not in dico:
                dico[k[0]] = {}
            dico[k[0]][k[1]] = v
    Lout.append(dico)
dico_filtered = {}
for n in new_df["NUM_POINTS"].unique():
    for nClust in new_df["NUM_CLUST"].unique():
        for impr_class in new_df["IMPR_CLASS"].unique():
            tmp_dico = new_df.query(f"NUM_POINTS == {n} and NUM_CLUST == {nClust} and IMPR_CLASS == '{impr_class}' and source == 'computed'").to_dict(orient="records")
            if len(tmp_dico) == 0:
                continue
            dico_filtered[f"{n}_{impr_class}_{nClust}_computed"] = tmp_dico
            
dico_out = {"grouped":Lout,"filtered":dico_filtered}

with open("data/table1_clust_visualize.json","w") as f:
    json.dump(dico_out,f,indent=2)

### Recreate table publication

In [80]:
data = {
    ('key',''):[],
    ('NUM_POINTS',''):[],
    ('NUM_CLUST',''):[],
    ('Obj. function values','BICL'):[],
    ('Obj. function values','FICL'):[],
    ('CPU times','BICL'):[],
    ('CPU times','FICL'):[],
    ('# of iterations','BICL'):[],
    ('# of iterations','FICL'):[],
}
nPts = df_table1["NUM_POINTS"].unique()
nClust = df_table1["NUM_CLUST"].unique()
imprClass = df_table1["IMPR_CLASS"].unique()
attribute = [["final_cost",'Obj. function values'],["duration",'CPU times'],["num_iter",'# of iterations']]
for npt,nclust,impr in product(nPts,nClust,imprClass):
    tmp_df = df_table1.query(f"NUM_POINTS == {npt} and NUM_CLUST == {nclust} and IMPR_CLASS == '{impr}'")
    try:
        dico = tmp_df.to_dict(orient="records")[0]
    except Exception:
        continue
    key = f"{npt}_{nclust}"
    if key not in data[('key','')]:
        data[('key','')].append(key)
        data[('NUM_POINTS','')].append(npt)
        data[('NUM_CLUST','')].append(nclust)
    for name,label in attribute:
        data[(label,impr)].append(dico[name])
pd.set_option('display.max_rows', None)
df_as_paper = pd.DataFrame(data)
df_as_paper = df_as_paper[[('NUM_POINTS',''),('NUM_CLUST',''),('Obj. function values','BICL'),('Obj. function values','FICL'),('CPU times','BICL'),('CPU times','FICL'),('# of iterations','BICL'),('# of iterations','FICL')]]
display(df_as_paper)
pd.set_option('display.max_rows', 20)

NUM_POINTS NUM_CLUST Obj. function values           CPU times            \
                                         BICL      FICL      BICL      FICL   
0           20         2                499.5    1499.5  0.016600  0.016672   
1           20         4               2499.5    3499.5  0.006039  0.005956   
2           20         8               4499.5    5499.5  0.001778  0.001876   
3           20        16               6499.5    7499.5  0.000197  0.000242   
4           30         2              16499.5   17499.5  0.026759  0.026769   
5           30         4              18499.5   19499.5  0.010216  0.010048   
6           30         8              20499.5   21499.5  0.003693  0.003681   
7           30        16              22499.5   23499.5  0.000886  0.000952   
8           40         2              32499.5   33499.5  0.036687  0.036826   
9           40         4              34499.5   35499.5  0.014335  0.014250   
10          40         8              36499.5   37499.5  0.005690  0.005609   
11          40        16              38499.5   39499.5  0.001723  0.001788   
12          40        32              40499.5   41499.5  0.000190  0.000237   
13          50         2              48499.5   49499.5  0.046602  0.046919   
14          50         4              50499.5   51499.5  0.018605  0.018382   
15          50         8              52499.5   53499.5  0.007746  0.007672   
16          50        16              54499.5   55499.5  0.002593  0.002645   
17          50        32              56499.5   57499.5  0.000504  0.000571   
18          60         2              64499.5   65499.5  0.057139  0.057162   
19          60         4              66499.5   67499.5  0.022690  0.022503   
20          60         8              68499.5   69499.5  0.009801  0.009576   
21          60        16              70499.5   71499.5  0.003527  0.003558   
22          60        32              72499.5   73499.5  0.000854  0.000939   
23          70         2              80499.5   81499.5  0.067235  0.067084   
24          70         4              82499.5   83499.5  0.027001  0.026693   
25          70         8              84499.5   85499.5  0.011906  0.011609   
26          70        16              86499.5   87499.5  0.004521  0.004499   
27          70        32              88499.5   89499.5  0.001233  0.001329   
28          70        64              90499.5   91499.5  0.000062  0.000086   
29          80         2              96499.5   97499.5  0.077430  0.077295   
30          80         4              98499.5   99499.5  0.031166  0.030835   
31          80         8             100499.5  101499.5  0.014029  0.013764   
32          80        16             102499.5  103499.5  0.005497  0.005423   
33          80        32             104499.5  105499.5  0.001647  0.001729   
34          80        64             106499.5  107499.5  0.000187  0.000233   
35          90         2             112499.5  113499.5  0.087531  0.087590   
36          90         4             114499.5  115499.5  0.035357  0.034898   
37          90         8             116499.5  117499.5  0.016099  0.015736   
38          90        16             118499.5  119499.5  0.006466  0.006395   
39          90        32             120499.5  121499.5  0.002085  0.002149   
40          90        64             122499.5  123499.5  0.000332  0.000395   
41         100         2             128499.5  129499.5  0.097624  0.097723   
42         100         4             130499.5  131499.5  0.039478  0.038970   
43         100         8             132499.5  133499.5  0.018195  0.017941   
44         100        16             134499.5  135499.5  0.007495  0.007367   
45         100        32             136499.5  137499.5  0.002530  0.002567   
46         100        64             138499.5  139499.5  0.000488  0.000563   
47         110         2             144499.5  145499.5  0.108096  0.108902   
48         110         4             146499.5  147499.